In [1]:
import numpy as np
import tensorflow as tf

In [2]:
tf.logging.set_verbosity(tf.logging.INFO)

In [7]:
def cnn_model_fn(features, labels, mode):
    # input layer
    #reshape( ~, -1) | -1 is wildcard number.
    input_layer = tf.reshape(features["x"],  [-1, 28, 28, 1])
    
    
    #　conv layer 1
    conv1 = tf.layers.conv2d(
        inputs = input_layer,
        # the number of filters a.k.a the dim of output space
        filters = 32,
        # filter size
        kernel_size = [5, 5],
        # either valid or same #same match output size with input size.
        padding= "same", 
        activation = tf.nn.relu)
    
    # pooling layer 1
    # can specify pool_size=2, if both int are same.
    pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=[2,  2], strides=2)
    
    # convolutional  layer 2
    conv2 = tf.layers.conv2d(
        inputs=pool1,
        filters=64,
        kernel_size=[5, 5],
        padding="same",
        activation=tf.nn.relu)
    # pooling layer 2
    pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[2, 2], strides=2)
    
    #reshape pooling layer to 2dim layer and connect to dense layer
    pool2_flat = tf.reshape(pool2, [-1, 7 * 7 * 64])
    dense = tf.layers.dense(inputs=pool2_flat, units=1024, activation=tf.nn.relu)
    
    #Dropout: A Simple Way to Prevent Neural Networks from Overfitting
    #if mode is TRAIN mode, dropout can run
    dropout = tf.layers.dropout(
        inputs=dense, rate=0.4, training=mode == tf.estimator.ModeKeys.TRAIN)
    
    # final layer is Logits Layer units number is 10 (0~9) for each target class 0~9
    logits = tf.layers.dense(inputs=dropout, units=10)
    
    predictions = {
        # argmax make the index of largest value and dont use deprecated dim but axis
        "classes": tf.argmax(input=logits, axis=1),
        # derive probabilities from logits layer using softmax act-func
        "probabilities": tf.nn.softmax(logits, name="softmax_tensor")
    }
    
    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)
    
    #next line is definition of loss
    
    
    #tf.onehot returns a one-hot tensor.
    #The locations represented by indices in indices take values take value on_value,
    #while all other locations take value off_value.
    onehot_labels = tf.one_hot(indices=tf.cast(labels, tf.int32), depth=10)
    #compute cross-entropy between onehot_labels and logits
    loss = tf.losses.softmax_cross_entropy(
        onehot_labels=onehot_labels, logits=logits)
    
    
    #configure the train Op in next line
    if mode == tf.estimator.ModeKeys.TRAIN:
        optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001)
        train_op = optimizer.minimize(
            loss=loss,
            #global_step parameter is to count the number of training steps 
            #that have been processed(to know when to end a training run).
            global_step=tf.train.get_global_step())
        return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)

    eval_metric_ops = {
            "accuracy" : tf.metrics.accuracy(
                labels=labels, predictions=predictions["classes"])
    }
    
    return tf.estimator.EstimatorSpec(
        mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)

In [4]:
mnist = tf.contrib.learn.datasets.load_dataset("mnist")
train_data = mnist.train.images
train_labels = np.asarray(mnist.train.labels, dtype=np.int32)
eval_data = mnist.test.images
eval_labels = np.asarray(mnist.test.labels, dtype=np.int32)


Instructions for updating:
Use the retry module or similar alternatives.
Instructions for updating:
Please use tf.data.
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST-data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST-data/train-labels-idx1-ubyte.gz
Extracting MNIST-data/t10k-images-idx3-ubyte.gz
Extracting MNIST-data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [20]:
# Create the Estimator
mnist_classifier = tf.estimator.Estimator(
    model_fn=cnn_model_fn, model_dir="/tmp/mnist_cnn_model")
    
# Set up logging for predictions
tensors_to_log = {"probabilities": "softmax_tensor"} 
logging_hook = tf.train.LoggingTensorHook(
    tensors=tensors_to_log, every_n_iter=100)


    
    

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/mnist_cnn_model', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x191f081be0>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [21]:
train_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": train_data},
    y=train_labels,
    batch_size=100,        
    num_epochs=None,
    shuffle=True)
mnist_classifier.train(
    input_fn=train_input_fn,
    steps=20000,
    hooks=[logging_hook])


INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/mnist_cnn_model/model.ckpt-21004
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 21005 into /tmp/mnist_cnn_model/model.ckpt.
INFO:tensorflow:probabilities = [[ 0.00000157  0.00000001  0.00000054  0.00000097  0.00000044  0.00000089
   0.          0.99985588  0.00000004  0.00013961]
 [ 0.          0.00000704  0.99987698  0.00010125  0.          0.          0.
   0.00000108  0.00001362  0.        ]
 [ 0.00001043  0.99734724  0.00012034  0.00013095  0.00066017  0.00002328
   0.0002922   0.00047086  0.00080249  0.00014204]
 [ 0.00057103  0.0000036   0.0000434   0.00045527  0.00006011  0.00035186
   0.00012882  0.00000674  0.99525511  0.00312414]
 [ 0.00000011  0.00000001  0.00000006  0.00000178  0.00000004  0.99991441
  

INFO:tensorflow:loss = 0.0424314, step = 21005
INFO:tensorflow:Loss for final step: 0.0424314.


In [22]:
# Evaluate the model and print results
eval_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x" : eval_data},
    y=eval_labels,
    num_epochs=1,
    shuffle=False)
eval_results = mnist_classifier.evaluate(input_fn=eval_input_fn)
print(eval_results)


INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-05-10-11:46:39
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/mnist_cnn_model/model.ckpt-21005
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2018-05-10-11:46:46
INFO:tensorflow:Saving dict for global step 21005: accuracy = 0.9714, global_step = 21005, loss = 0.0932526
{'accuracy': 0.97140002, 'loss': 0.093252629, 'global_step': 21005}
